# CS4248 Project Group 23

In [ ]:
# If you wish to run this on Google Colab, mount the Google drive by running this cell or click the `files` icon on the left navbar
# and click mount Google Drive (it takes some time to load)
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/My Drive/<The path to this notebook in your Google Drive>"
!cd "/content/drive/My Drive/<The path to this notebook in your Google Drive>"

In [1]:
import pandas as pd
from sklearn.metrics import f1_score
import numpy as np

In [4]:
# Unzip raw_data.zip locally
import zipfile
with zipfile.ZipFile('raw_data.zip', 'r') as zip_ref:
    zip_ref.extractall()

Feature Engineering: Capture various features of the text (e.g. punctuation, stopwords, statement length). 
Test out different tokenizers to capture their performance.


In [2]:
import gensim.downloader as api

def load_glove_model():
    glove_model = api.load('glove-wiki-gigaword-300')
    print("Done.",len(glove_model)," words loaded!")
    return glove_model, glove_model.vector_size

glove_model, glove_dim = load_glove_model()

Done. 400000  words loaded!


In [3]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# todo parallelize this in future


def preprocess_text(text):
    # Lowercase text
    text = text.lower()
    # Remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # tokenize
    words = text.split()
    # Get the GloVe vectors
    vectors = [glove_model[word] for word in words if word in glove_model]
    # If vectors is empty, return a vector of zeros
    if not vectors:
        print("No vectors found for the text: ", text)
        return np.zeros(glove_model.vector_size)
    return np.mean(vectors, axis=0)

In [4]:
train = pd.read_csv("./raw_data/fulltrain.csv")
x_train = np.array([preprocess_text(text) for text in train.iloc[:, 1]]) 
y_train = train.iloc[:, 0].values - 1 # subtract 1 to make the labels 0-based

No vectors found for the text:  abfs daksl 
No vectors found for the text:  nsas xkeyscore explainedyoutube 


Test out different kinds of models and find the most effective architectures.|

In [5]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=10000)
model.fit(x_train, y_train)
y_pred = model.predict(x_train)


In [11]:
# load a previous model
import joblib
model = joblib.load("./sklearn_models/logisticRegression.pkl")
y_pred = model.predict(x_train)

Perform hyperparameter tuning on best 3 models.

In [ ]:
from sklearn.model_selection import GridSearchCV

# or perform hyperparameter tuning
# Create the hyperparameters grid
param_grid = {

}

grid_search = GridSearchCV(())

# Train
grid_search.fit(x_train, y_train)

print(grid_search.best_params_)

# Use best model
model = grid_search.best_estimator_

In [6]:
# get the training error
f1_score(y_train, y_pred, average='macro')
# print for the train set f1 score is
print("Train Set: " + str(f1_score(y_train, y_pred, average='macro')))

Train Set: 0.8681960038153329


In [7]:
# get the prediction for the test set
test = pd.read_csv('./raw_data/balancedtest.csv')
X_test = np.array([preprocess_text(text) for text in test.iloc[:, 1]]) 
y_test = test.iloc[:, 0].values - 1 # subtract 1 to make the labels 0-based
y_pred = model.predict(X_test)

# get the f1 score
f1_score(y_test, y_pred, average='macro')

0.6620506433511602

In [10]:
# save the sklearn model
import joblib
joblib.dump(model, './sklearn_models/logisticRegression.pkl')

['./sklearn_models/logisticRegression.pkl']